# This nb is to prototype and compare probes


In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

plt.style.use("ggplot")

from typing import Optional, List, Dict, Union
from jaxtyping import Float
from torch import Tensor

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

from pathlib import Path
from einops import rearrange


import datasets
from datasets import Dataset

from loguru import logger

logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")

# # quiet please
torch.set_float32_matmul_precision("medium")
import warnings
warnings.filterwarnings("ignore", ".*does not have many workers.*")


In [2]:
# load my code
%load_ext autoreload
%autoreload 2

import lightning.pytorch as pl
from src.datasets.dm import DeceptionDataModule
from src.models.pl_lora_ft import AtapterFinetuner

from src.config import ExtractConfig
from src.prompts.prompt_loading import load_preproc_dataset, load_preproc_datasets
from src.models.load import load_model
from src.helpers.torch_helpers import clear_mem
from src.models.phi.model_phi import PhiForCausalLMWHS


# Load previously made datasets of hidden states

In [9]:
!ls -altrh '/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.ds/'

!ls -altrh '/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.ds/ds_valtest_8c031b4aa03ae4d2'


total 1.3G
drwxrwxr-x 9 wassname wassname 4.0K Dec 23 18:17 ..
-rw-rw-r-- 1 wassname wassname 209M Dec 23 18:51 ds_OOD_6e05f2126512ce02
-rw-rw-r-- 1 wassname wassname 209M Dec 23 18:54 ds_valtest_a295cab1f3d786a1
-rw-rw-r-- 1 wassname wassname 209M Dec 24 06:45 ds_OOD_89ad9d7b591f3c44
-rw-rw-r-- 1 wassname wassname 209M Dec 24 06:49 ds_valtest_7777409fa7e08370
-rw-rw-r-- 1 wassname wassname 209M Dec 24 07:45 ds_OOD_88214ab52bf8f023
drwxrwxr-x 2 wassname wassname 4.0K Dec 24 07:45 .
-rw-rw-r-- 1 wassname wassname 209M Dec 24 07:49 ds_valtest_8c031b4aa03ae4d2
-rw-rw-r-- 1 wassname wassname 209M Dec 24 07:49 /media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.ds/ds_valtest_8c031b4aa03ae4d2


In [17]:
f1_ood = '/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.ds/ds_OOD_88214ab52bf8f023'
f1_val = '/media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/.ds/ds_valtest_8c031b4aa03ae4d2'


In [29]:
ds_val = Dataset.from_file(f1_val).with_format("torch")

ds_oos = Dataset.from_file(f1_ood).with_format("torch")
ds_out = ds_oos
ds_oos


Dataset({
    features: ['end_logits_base', 'choice_probs_base', 'binary_ans_base', 'label_true_base', 'label_instructed_base', 'instructed_to_lie_base', 'sys_instr_name_base', 'example_i_base', 'ds_string_base', 'template_name_base', 'correct_truth_telling_base', 'correct_instruction_following_base', 'end_residual_stream_base', 'end_logits_adapt', 'choice_probs_adapt', 'binary_ans_adapt', 'label_true_adapt', 'label_instructed_adapt', 'instructed_to_lie_adapt', 'sys_instr_name_adapt', 'example_i_adapt', 'ds_string_adapt', 'template_name_adapt', 'correct_truth_telling_adapt', 'correct_instruction_following_adapt', 'end_residual_stream_adapt'],
    num_rows: 201
})

# Probes

## Linear


In [30]:
from src.eval.interventions import check_lr_intervention_predictive
from src.eval.labels import ranking_truth_telling
from src.eval.ds import filter_ds_to_known
ds_known = filter_ds_to_known(ds_out, verbose=True)
hs_normal = ds_out['end_residual_stream_base']
hs_intervene = ds_out['end_residual_stream_adapt']
label_fn = ranking_truth_telling
hs = hs_normal - hs_intervene
y = label_fn(ds_out)


check_lr_intervention_predictive(hs, y, verbose=True)


select rows are 45.83% based on knowledge
          0         1
0  0.724138  0.275862
1  0.279070  0.720930
              precision  recall  f1-score  support
0                  0.78    0.72      0.75       58
1                  0.66    0.72      0.69       43
accuracy           0.72    0.72      0.72      101
macro avg          0.72    0.72      0.72      101
weighted avg       0.73    0.72      0.72      101


0.7538091419406575

# Conv

# Bayesian

# Quantile